# Cleaning Up

This notebook demonstrates how to clean up all the resources created in the previous set of notebooks.

In [1]:
import boto3
import json

In [2]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In [9]:
%store -r

Please note these steps rely on each step being fully completed before the other will work. If you receive an error, wait a minute or two and try again.

## Clean up Campaigns

Start by cleaning up the campaigns. You need to delete campaigns before you can delete the solution versions upon which they are based. 

The code below will delete the Campaigns you created.

In [4]:
personalize.delete_campaign(
    campaignArn = userpersonalization_campaign_arn
)
personalize.delete_campaign(
    campaignArn = sims_campaign_arn
)
personalize.delete_campaign(
    campaignArn = rerank_campaign_arn
)

{'ResponseMetadata': {'RequestId': '218a6132-2e8f-4910-bc1e-628904b4e30c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Fri, 04 Sep 2020 13:31:04 GMT',
   'x-amzn-requestid': '218a6132-2e8f-4910-bc1e-628904b4e30c',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up event trackers

Next, clean up any event trackers. The code below will delete your event tracker.

In [5]:
personalize.delete_event_tracker(
    eventTrackerArn = event_tracker_arn
)

{'ResponseMetadata': {'RequestId': '1529fb40-ec82-4aca-a36b-0f0935fd7bbf',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Fri, 04 Sep 2020 13:32:59 GMT',
   'x-amzn-requestid': '1529fb40-ec82-4aca-a36b-0f0935fd7bbf',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up Filters

Next, clean up any filters. The code below iterate over all filters you created and delete them. Note, if you get an error about a campaign, that is fine, wait for 2 minutes then try the cell below again. 

In [8]:
for filter_arn in meta_filter_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )
for filter_arn in interaction_filter_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )

## Clean up solutions

Next, clean up the solutions. The code below will delete the solutions that were created.

In [10]:
personalize.delete_solution(
    solutionArn = user_personalization_solution_arn
)
personalize.delete_solution(
    solutionArn = sims_solution_arn
)
personalize.delete_solution(
    solutionArn = rerank_solution_arn
)

{'ResponseMetadata': {'RequestId': '15f45fa7-00f2-47a7-b47a-bc5bd231d076',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Fri, 04 Sep 2020 13:42:01 GMT',
   'x-amzn-requestid': '15f45fa7-00f2-47a7-b47a-bc5bd231d076',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up datasets

Next, clean up the datasets. The code below will delete the item and interactions datasets.

In [11]:
personalize.delete_dataset(
    datasetArn = items_dataset_arn
)
personalize.delete_dataset(
    datasetArn = interactions_dataset_arn
)

{'ResponseMetadata': {'RequestId': '2fa9a131-9676-4a97-a206-ea7525abfbc9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Fri, 04 Sep 2020 13:43:18 GMT',
   'x-amzn-requestid': '2fa9a131-9676-4a97-a206-ea7525abfbc9',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up the schemas

Next, clean up the schemas. You create an interaction and item schema, delete them.

In [13]:
personalize.delete_schema(
    schemaArn = interaction_schema_arn
)
personalize.delete_schema(
    schemaArn = itemmetadataschema_arn
)

{'ResponseMetadata': {'RequestId': '98677d2b-2d31-446f-bf97-ec0fdc6f0c42',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Fri, 04 Sep 2020 13:44:57 GMT',
   'x-amzn-requestid': '98677d2b-2d31-446f-bf97-ec0fdc6f0c42',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up the dataset groups

Finally, clean up the dataset group:

In [12]:
personalize.delete_dataset_group(
    datasetGroupArn = dataset_group_arn
)

{'ResponseMetadata': {'RequestId': 'e91143e1-d8bb-4597-adc5-4ccbca189c22',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Fri, 04 Sep 2020 13:44:52 GMT',
   'x-amzn-requestid': 'e91143e1-d8bb-4597-adc5-4ccbca189c22',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up the S3 bucket and IAM role

Start by deleting the role, then empty the bucket, then delete the bucket.

In [14]:
iam = boto3.client('iam')

Identify the name of the role you want to delete.

You cannot delete an IAM role which still has policies attached to it. So after you have identified the relevant role, let's list the attached policies of that role.

In [15]:
iam.list_attached_role_policies(
    RoleName = role_name
)

{'AttachedPolicies': [{'PolicyName': 'AmazonS3FullAccess',
   'PolicyArn': 'arn:aws:iam::aws:policy/AmazonS3FullAccess'},
  {'PolicyName': 'AmazonPersonalizeFullAccess',
   'PolicyArn': 'arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess'}],
 'IsTruncated': False,
 'ResponseMetadata': {'RequestId': '08d25233-1aec-496b-8bfc-6e66f182369b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '08d25233-1aec-496b-8bfc-6e66f182369b',
   'content-type': 'text/xml',
   'content-length': '728',
   'date': 'Fri, 04 Sep 2020 13:45:33 GMT'},
  'RetryAttempts': 0}}

You need to detach the policies in the result above using the code below. Repeat for each attached policy.

In [16]:
iam.detach_role_policy(
    RoleName = role_name,
    PolicyArn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
)

{'ResponseMetadata': {'RequestId': '70f0daf9-3811-4ae7-b5c7-16962833e643',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '70f0daf9-3811-4ae7-b5c7-16962833e643',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Fri, 04 Sep 2020 13:45:35 GMT'},
  'RetryAttempts': 0}}

In [17]:
iam.detach_role_policy(
    RoleName = role_name,
    PolicyArn = 'arn:aws:iam::aws:policy/AmazonS3FullAccess'
)

{'ResponseMetadata': {'RequestId': 'ecd7b7bf-42eb-457f-b7cd-105ff8fa9452',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ecd7b7bf-42eb-457f-b7cd-105ff8fa9452',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Fri, 04 Sep 2020 13:45:36 GMT'},
  'RetryAttempts': 0}}

Finally, you should be able to delete the IAM role.

In [18]:
iam.delete_role(
    RoleName = role_name
)

{'ResponseMetadata': {'RequestId': 'ab35cdde-99e6-450b-9a32-9d8c7c932fbc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ab35cdde-99e6-450b-9a32-9d8c7c932fbc',
   'content-type': 'text/xml',
   'content-length': '200',
   'date': 'Fri, 04 Sep 2020 13:45:38 GMT'},
  'RetryAttempts': 0}}

To delete an S3 bucket, it first needs to be empty. The easiest way to delete an S3 bucket, is just to navigate to S3 in the AWS console, delete the objects in the bucket, and then delete the S3 bucket itself.